<a href="https://colab.research.google.com/github/NormLorenz/ai-llm-openai-mcp/blob/main/openai-mcp.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Using OpenAI with a MCP Server

In [1]:
# Install required packages

!pip install --upgrade pip
!pip install fastmcp openai nest_asyncio

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 15.8 MB/s eta 0:00:00
  Attempting uninstall: pip
    Found existing installation: pip 24.1.2
    Uninstalling pip-24.1.2:
      Successfully uninstalled pip-24.1.2
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 57.6 MB/s  0:00:00
  Attempting uninstall: wrapt
    Found existing installation: wrapt 2.0.1
    Uninstalling wrapt-2.0.1:
      Successfully uninstalled wrapt-2.0.1
  Attempting uninstall: referencing
    Found existing installation: referencing 0.37.0
    Uninstalling referencing-0.37.0:
      Successfully uninstalled referencing-0.37.0
  Attempting uninstall: opentelemetry-api
    Found existing installation: opentelemetry-api 1.37.0
    Uninstalling opentelemetry-api-1.37.0:
      Successfully uninstalled opentelemetry-api-1.37.0
  Attempting uninstall: opentelemetry-semantic-conventions
    Found existing installation: opentelemetry-semantic-conventions 0.58b0
    Uninstalling opentelemetry-semantic-convention

This code cell sets up and starts a FastMCP server. It defines several functionalities: Tools like get_forecast, get_alerts, multiply, and health_check which perform specific actions. It also registers Resources such as climate_data, get_greeting, and get_config to provide data. Additionally, it includes a Prompt called analyze_data for generating structured prompts. The server is then launched in a background thread to make these tools and resources accessible via HTTP, specifically configured for notebook compatibility.

In [2]:
# The MCP Server

from fastmcp import FastMCP
import nest_asyncio
import threading
import time

nest_asyncio.apply()

mcp = FastMCP(
    name="WeatherServer",
    instructions="This provides an up to date weather forecast for any location."
)

# Tool 1: Forecast
@mcp.tool("get_forecast")
def get_forecast(location: str):
    """Returns a forecast for the given location."""
    return {"forecast": f"Sunny in {location}"}

# Tool 2: Alerts
@mcp.tool(name="get_alerts")
def get_alerts(location: str):
    """Returns any severe weather alerts for the given location."""
    return {"alerts": f"No severe alerts currently for {location}"}

# Tool 3: Math
@mcp.tool
def multiply(a: float, b: float) -> float:
    """Multiplies two numbers together."""
    return a * b

# Tool 4: Health Check
@mcp.tool
def health_check():
    """Returns the health status of the server."""
    return {"status": "ok"}

# Resource 1: Climate Data
@mcp.resource(uri="resource://climate")
def climate_data():
    """Return static climate information."""
    return {
        "Berlin": {"avg_temp": "10°C", "rainfall": "570mm"},
        "Boise": {"avg_temp": "12°C", "rainfall": "300mm"},
        "Tokyo": {"avg_temp": "16°C", "rainfall": "1500mm"}
    }

# Resource 2: Basic dynamic resource returning a string
@mcp.resource("resource://greeting")
def get_greeting() -> str:
    """Provides a simple greeting message."""
    return "Hello from FastMCP Resources!"

# Resource 3: Resource returning JSON data (dict is auto-serialized)
@mcp.resource("data://config")
def get_config() -> dict:
    """Provides application configuration as JSON."""
    return {
        "theme": "dark",
        "version": "1.2.0",
        "features": ["tools", "resources"],
    }

# Resource 4: Resource returning secret data
@mcp.resource("data://secret", enabled=False)
def get_secret_data():
    """This resource is currently disabled."""
    return "Secret data"

# Prompt 1: Analysis of Numerical data
@mcp.prompt
def analyze_data(data_points: list[float]) -> str:
    """Creates a prompt asking for analysis of numerical data."""
    formatted_data = ", ".join(str(point) for point in data_points)
    return f"Please analyze these data points: {formatted_data}"

# Define the function to run the server
def run_server():
    # Use transport="streamable-http" for compatibility with notebooks/Colab
    print("🚀 Starting FastMCP server in background thread...")
    mcp.run(transport="streamable-http", host="0.0.0.0", port=8000)

# Start the server in a separate thread
server_thread = threading.Thread(target=run_server, daemon=True)
server_thread.start()

# Give the server a moment to start up
time.sleep(5)
print("✅ Server should be running. Access it at http://localhost:8000/mcp")

🚀 Starting FastMCP server in background thread...


/usr/local/lib/python3.12/dist-packages/jupyter_client/session.py:203: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  return datetime.utcnow().replace(tzinfo=utc)


╭──────────────────────────────────────────────────────────────────────────────╮                  
                 │                                                                              │                  
                 │                         ▄▀▀ ▄▀█ █▀▀ ▀█▀ █▀▄▀█ █▀▀ █▀█                        │                  
                 │                         █▀  █▀█ ▄▄█  █  █ ▀ █ █▄▄ █▀▀                        │                  
                 │                                                                              │                  
                 │                                FastMCP 2.14.1                                │                  
                 │                                                                              │                  
                 │                                                                              │                  
                 │                   🖥  Server name: WeatherServer                              │                  
                 │                                                                              │                  
                 │                   📦 Transport:   HTTP                                       │                  
                 │                   🔗 Server URL:  http://0.0.0.0:8000/mcp                    │                  
                 │                                                                              │                  
                 │                   📚 Docs:        https://gofastmcp.com                      │                  
                 │                   🚀 Hosting:     https://fastmcp.cloud                      │                  
                 │                                                                              │                  
                 ╰──────────────────────────────────────────────────────────────────────────────╯

[12/20/25 14:27:46] INFO     Starting MCP server 'WeatherServer' with transport 'streamable-http' on ]8;id=217362;file:///usr/local/lib/python3.12/dist-packages/fastmcp/server/server.py\server.py]8;;\:]8;id=917905;file:///usr/local/lib/python3.12/dist-packages/fastmcp/server/server.py#2618\2618]8;;\
                             http://0.0.0.0:8000/mcp                                                               

INFO:     Started server process [1009]
INFO:     Waiting for application startup.
INFO:     Application startup complete.
INFO:     Uvicorn running on http://0.0.0.0:8000 (Press CTRL+C to quit)


✅ Server should be running. Access it at http://localhost:8000/mcp


This code below sets up a test suite for the FastMCP server you started in the previous cell. It uses the fastmcp.Client to connect to the server and test its various functionalities: calling tools like get_forecast and multiply, reading resources such as climate_data, and retrieving prompts like analyze_data. The code then prints whether each test passed or failed, along with a summary of all tests. The output indicates that the client wasn't connected, which is a common issue when using fastmcp.Client outside of its recommended async with context manager.

In [ ]:
# Create a MCP server test suite

import asyncio
from fastmcp import Client

# MCP_SERVER_URL is available from previous cells in the kernel state.
MCP_SERVER_URL = "http://localhost:8000/mcp"

async def run_mcp_client_tests(server_url: str):
    print(f"🚀 Starting FastMCP client tests against: {server_url}")
    print("Ensure the MCP server is running in a separate thread/process.")

    all_results = []

    async with Client(server_url) as client:
        print("\n--- Inspecting Client Object ---")
        print(f"Client type: {type(client)}")
        print(f"Client dir: {dir(client)}")
        print("--------------------------------")

        # --- Test Tools ---
        print("\n--- Running Tool Tests ---")
        tool_test_cases = [
            {'name': 'get_forecast', 'arguments': {'location': 'London'}, 'expected_result': {"forecast": "Sunny in London"}},
            {'name': 'get_alerts', 'arguments': {'location': 'London'}, 'expected_result': {"alerts": "No severe alerts currently for London"}},
            {'name': 'multiply', 'arguments': {'a': 5, 'b': 3}, 'expected_result': 15.0},
            {'name': 'health_check', 'arguments': {}, 'expected_result': {"status": "ok"}}
        ]

        for test_case in tool_test_cases:
            name = test_case['name']
            args = test_case['arguments']
            expected = test_case['expected_result']
            print(f"Testing tool: {name} with args: {args}")
            try:
                result = await client.call_tool(name, arguments=args)
                passed = (result.data == expected)
                all_results.append({
                    "type": "tool",
                    "name": name,
                    "arguments": args,
                    "passed": passed,
                    "result": result,
                    "expected": expected
                })
                print(f"  {'✅ PASSED' if passed else '❌ FAILED'}: Result={result}")
            except Exception as e:
                all_results.append({
                    "type": "tool",
                    "name": name,
                    "arguments": args,
                    "passed": False,
                    "error": str(e)
                })
                print(f"  ❌ FAILED: Error={e}")

        # --- Test Resources ---
        print("\n--- Running Resource Tests ---")
        resource_test_cases = [
            {'name': 'climate', 'expected_result': {
                "Berlin": {"avg_temp": "10°C", "rainfall": "570mm"},
                "Boise": {"avg_temp": "12°C", "rainfall": "300mm"},
                "Tokyo": {"avg_temp": "16°C", "rainfall": "1500mm"}
            }}
        ]

        for test_case in resource_test_cases:
            name = test_case['name']
            expected = test_case['expected_result']
            print(f"Testing resource: {name}")
            try:
                result = await client.read_resource(name)
                passed = (result == expected)
                all_results.append({
                    "type": "resource",
                    "name": name,
                    "passed": passed,
                    "result": result,
                    "expected": expected
                })
                print(f"  {'✅ PASSED' if passed else '❌ FAILED'}: Result={result}")
            except Exception as e:
                all_results.append({
                    "type": "resource",
                    "name": name,
                    "passed": False,
                    "error": str(e)
                })
                print(f"  ❌ FAILED: Error={e}")

        # --- Test Prompts ---
        print("\n--- Running Prompt Tests ---")
        prompt_test_cases = [
            {'name': 'analyze_data', 'arguments': {'data_points': [10.5, 20.1, 30.0]}, 'expected_prefix': "Please analyze these data points: 10.5, 20.1, 30.0"}
        ]

        for test_case in prompt_test_cases:
            name = test_case['name']
            args = test_case['arguments']
            expected_prefix = test_case['expected_prefix']
            print(f"Testing prompt: {name} with args: {args}")
            try:
                result = await client.get_prompt(name, arguments=args)
                # Correctly access the text content of the prompt message
                message = result.messages[0]
                passed = (message.content.text == expected_prefix)
                all_results.append({
                    "type": "prompt",
                    "name": name,
                    "arguments": args,
                    "passed": passed,
                    "result": result,
                    "expected": expected_prefix
                })
                print(f"  {'✅ PASSED' if passed else '❌ FAILED'}: Result='{message.content.text}'")
            except Exception as e:
                all_results.append({
                    "type": "prompt",
                    "name": name,
                    "arguments": args,
                    "passed": False,
                    "error": str(e)
                })
                print(f"  ❌ FAILED: Error={e}")

    print("\n--- Test Summary ---")
    total_tests = len(all_results)
    passed_tests = sum(1 for r in all_results if r.get('passed', False))
    failed_tests = total_tests - passed_tests

    for res in all_results:
        status = 'PASSED' if res.get('passed', False) else 'FAILED'
        print(f"[{res['type'].upper()}] {res['name']}: {status}")
        if not res.get('passed', False):
            if 'error' in res:
                print(f"  Error: {res['error']}")
            else:
                print(f"  Expected: {res.get('expected')}")
                print(f"  Got: {res.get('result')}")
    print(f"\nTotal Tests: {total_tests}, Passed: {passed_tests}, Failed: {failed_tests}")

if __name__ == "__main__":
    # Make sure to run the MCP server cell (fYO0LykYQ05Y) before running this test suite.
    asyncio.run(run_mcp_client_tests(MCP_SERVER_URL))

This code sets up an OpenAI agent to interact with your FastMCP server. It initializes an OpenAI client, defines the get_forecast tool in a format OpenAI understands, and then sends a user query about the weather to OpenAI. If OpenAI decides to use the get_forecast tool, the code extracts the tool's arguments, calls the corresponding tool on your FastMCP server, and then sends the tool's output back to OpenAI to generate a natural language response to the original query.

In [4]:
from openai import OpenAI
import asyncio
from fastmcp import Client
from google.colab import userdata

# Print the key prefixes to help with any debugging
openai_api_key = userdata.get("OPENAI_API_KEY")

if openai_api_key:
    print(f"OpenAI API Key exists and begins {openai_api_key[:8]}")
else:
    print("OpenAI API Key not set")

# Assign the key
openai = OpenAI(api_key=openai_api_key)

# Assuming MCP_SERVER_URL is defined from previous cells
MCP_SERVER_URL = "http://localhost:8000/mcp"

async def run_openai_agent():
    # Initialize OpenAI client (ensure OPENAI_API_KEY is set as an environment variable)
    # If you don't have an OpenAI API key, you can comment this out and observe
    # how the tool call is suggested by the model based on its structure.
    # For this example, we assume you have one configured.
    client_openai = openai.OpenAI()

    print(f"Connecting to MCP server at: {MCP_SERVER_URL}")

    async with Client(MCP_SERVER_URL) as mcp_client:
        # Define the tool for OpenAI in the required format
        tools = [
            {
                "type": "function",
                "function": {
                    "name": "get_forecast",
                    "description": "Returns a forecast for the given location.",
                    "parameters": {
                        "type": "object",
                        "properties": {
                            "location": {
                                "type": "string",
                                "description": "The city and state, e.g. San Francisco, CA"
                            }
                        },
                        "required": ["location"]
                    }
                }
            }
        ]

        print("\n--- OpenAI Agent Request ---")
        # Make a chat completion request to OpenAI, including the tool
        response = client_openai.chat.completions.create(
            model="gpt-3.5-turbo", # Or gpt-4, or other models that support tools
            messages=[
                {"role": "user", "content": "What's the weather like in London?"}
            ],
            tools=tools,
            tool_choice="auto", # Let OpenAI decide if it wants to call the tool
        )

        response_message = response.choices[0].message

        # Check if OpenAI decided to call a tool
        if response_message.tool_calls:
            tool_call = response_message.tool_calls[0]
            function_name = tool_call.function.name
            function_args = tool_call.function.arguments

            print(f"OpenAI wants to call tool: {function_name} with arguments: {function_args}")

            if function_name == "get_forecast":
                # Parse the arguments string to a dictionary
                import json
                args_dict = json.loads(function_args)
                location = args_dict.get("location")

                if location:
                    print(f"Calling MCP tool 'get_forecast' for location: {location}")
                    tool_response = await mcp_client.call_tool(function_name, arguments=args_dict)
                    print(f"MCP Tool Response: {tool_response.data}")

                    # Optionally, you can send the tool's output back to OpenAI for further processing
                    second_response = client_openai.chat.completions.create(
                        model="gpt-3.5-turbo",
                        messages=[
                            {"role": "user", "content": "What's the weather like in London?"},
                            response_message, # the assistant's previous message with tool_calls
                            {
                                "role": "tool",
                                "tool_call_id": tool_call.id,
                                "name": function_name,
                                "content": json.dumps(tool_response.data),
                            },
                        ],
                    )
                    print("\n--- OpenAI Final Response ---")
                    print(second_response.choices[0].message.content)
                else:
                    print("Error: 'location' not found in tool arguments.")
            else:
                print(f"OpenAI suggested an unknown tool: {function_name}")
        else:
            print("OpenAI did not suggest a tool call.")
            print(f"OpenAI's response: {response_message.content}")

# Run the agent
if __name__ == "__main__":
    # Ensure the MCP server cell (fYO0LykYQ05Y) has been executed
    # and you have an OpenAI API key configured as an environment variable.
    asyncio.run(run_openai_agent())


OpenAI API Key exists and begins sk-proj-


AttributeError: 'OpenAI' object has no attribute 'OpenAI'